In [37]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("suleymansulak/obesity-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/suleymansulak/obesity-dataset/versions/1


In [38]:
import os
os.listdir(path)

['Obesity_Dataset.xlsx',
 'Obesity_Dataset',
 'Obesity_Dataset.arff',
 '10.33484-sinopfbd.1445215-3764172.pdf']

In [39]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [40]:
df = pd.read_excel(f'{path}/Obesity_Dataset.xlsx')
df.head()

,Sex,Age,Height,Overweight_Obese_Family,Consumption_of_Fast_Food,Frequency_of_Consuming_Vegetables,Number_of_Main_Meals_Daily,Food_Intake_Between_Meals,Smoking,Liquid_Intake_Daily,Calculation_of_Calorie_Intake,Physical_Excercise,Schedule_Dedicated_to_Technology,Type_of_Transportation_Used,Class
0,2,18,155,2,2,3,1,3,2,1,2,3,3,4,2
1,2,18,158,2,2,3,1,1,2,1,2,1,3,3,2
2,2,18,159,2,2,2,1,3,2,3,2,2,3,4,2
3,2,18,162,2,2,2,2,2,2,2,2,1,3,4,2
4,2,18,165,2,1,2,1,3,2,1,2,3,3,2,2


In [41]:
df.shape

(1610, 15)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610 entries, 0 to 1609
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Sex                                1610 non-null   int64
 1   Age                                1610 non-null   int64
 2   Height                             1610 non-null   int64
 3   Overweight_Obese_Family            1610 non-null   int64
 4   Consumption_of_Fast_Food           1610 non-null   int64
 5   Frequency_of_Consuming_Vegetables  1610 non-null   int64
 6   Number_of_Main_Meals_Daily         1610 non-null   int64
 7   Food_Intake_Between_Meals          1610 non-null   int64
 8   Smoking                            1610 non-null   int64
 9   Liquid_Intake_Daily                1610 non-null   int64
 10  Calculation_of_Calorie_Intake      1610 non-null   int64
 11  Physical_Excercise                 1610 non-null   int64
 12  Schedule_Dedicated_t

In [43]:
df.head()

,Sex,Age,Height,Overweight_Obese_Family,Consumption_of_Fast_Food,Frequency_of_Consuming_Vegetables,Number_of_Main_Meals_Daily,Food_Intake_Between_Meals,Smoking,Liquid_Intake_Daily,Calculation_of_Calorie_Intake,Physical_Excercise,Schedule_Dedicated_to_Technology,Type_of_Transportation_Used,Class
0,2,18,155,2,2,3,1,3,2,1,2,3,3,4,2
1,2,18,158,2,2,3,1,1,2,1,2,1,3,3,2
2,2,18,159,2,2,2,1,3,2,3,2,2,3,4,2
3,2,18,162,2,2,2,2,2,2,2,2,1,3,4,2
4,2,18,165,2,1,2,1,3,2,1,2,3,3,2,2


In [44]:
df['Sex'].unique()

array([2, 1])

In [45]:
df['Class'].unique()

array([2, 3, 4, 1])

In [46]:
y = df['Class'].values
X = df.drop('Class', axis=1)

In [47]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(1610, 14)
(1610, 14)


In [48]:
# build mask
gender_0_mask = df['Sex'] == 2
gender_1_mask = df['Sex'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  898
Male:  712


In [49]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [50]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  1.000000  0.166667  0.833333  0.000000      44       1   
1      2  Female  1.000000  0.250000  0.750000  0.000000      41       1   
2      3  Female  1.000000  0.571429  0.428571  0.000000      39       4   
3      4  Female  1.000000  0.625000  0.375000  0.000000      54       5   
4      5  Female  0.978261  0.333333  0.666667  0.021739      45       2   
5      6  Female  0.942857  1.000000  0.000000  0.057143      33       4   
6      7  Female  0.979167  1.000000  0.000000  0.020833      47       2   
7      8  Female  1.000000  0.428571  0.571429  0.000000      38       3   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [51]:
results_df = pd.concat(results_list, ignore_index=True)

In [52]:
result_path = './results/K33_result.xlsx'
results_df.to_excel(result_path, index=False)

In [53]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,1.000000,0.166667,0.833333,0.000000,44,1,5,0,...,5,0,0.292683,1,0,0.707317,12,7,0,29
1,2,Female,1.000000,0.250000,0.750000,0.000000,41,1,3,0,...,1,1,0.210526,1,0,0.789474,8,4,0,30
2,3,Female,1.000000,0.571429,0.428571,0.000000,39,4,3,0,...,1,1,0.289474,1,0,0.710526,11,7,0,27
3,4,Female,1.000000,0.625000,0.375000,0.000000,54,5,3,0,...,2,1,0.270833,1,0,0.729167,13,8,0,35
4,5,Female,0.978261,0.333333,0.666667,0.021739,45,2,4,1,...,4,2,0.209302,1,0,0.790698,9,6,0,34


In [54]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(36.0), pvalue=np.float64(0.17957106951142032))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(8.0), pvalue=np.float64(0.0009958061785853432))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(67.0), pvalue=np.float64(0.10116105881500473))
DT -TPR: MannwhitneyuResult(statistic=np.float64(34.0), pvalue=np.float64(0.186009244408745))
DT - FPR: MannwhitneyuResult(statistic=np.float64(51.0), pvalue=np.float64(0.9683123957281493))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(72.5), pvalue=np.float64(0.06013978603218171))
RF -TPR: MannwhitneyuResult(statistic=np.float64(45.0), pvalue=np.float64(0.36812025069351895))
RF - FPR: MannwhitneyuResult(statistic=np.float64(56.0), pvalue=np.float64(0.6621297332852658))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(55.0), pvalue=np.float64(0.36812025069351895))
LR -TPR: MannwhitneyuResult(statistic=np.float64(47.5), pvalue=np.float64(0.8645050825863402))
LR - FPR: MannwhitneyuResult(